In [ ]:
import numpy as np
import pandas as pd

In [ ]:
X_train = pd.read_csv("../input/X_train.csv")
y_train = pd.read_csv("../input/y_train.csv")
sub = pd.read_csv("../input/sample_submission.csv")

In [ ]:
def flattenDataframe(df):
    '''
    'Flatten a dataframe
    '''
    df_new = pd.DataFrame([])
    for col in df.columns[3:]:
        df_new[col + '_mean'] = df.groupby(['series_id'])[col].mean()
        df_new[col + '_std'] = df.groupby(['series_id'])[col].std()
        df_new[col + '_var'] = df.groupby(['series_id'])[col].var()
        df_new[col + '_sem'] = df.groupby(['series_id'])[col].sem()
        df_new[col + '_max'] = df.groupby(['series_id'])[col].max()
        df_new[col + '_min'] = df.groupby(['series_id'])[col].min()
        df_new[col + '_max_to_min'] = df_new[col + '_max'] / df_new[col + '_min']
        df_new[col + '_max_minus_min'] = df_new[col + '_max'] - df_new[col + '_min']
        df_new[col + '_std_to_var'] = df_new[col + '_std'] * df_new[col + '_var']
        df_new[col + '_mean_abs_change'] = df.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df_new[col + '_abs_max'] = df.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
    return df_new

In [ ]:
X_train_flat = flattenDataframe(X_train)
X_train_flat.head()

### Apply Standard Scaling to our train dataset

When using Neural Network and Deep Learning based systems, it is usually a good idea to Standardize your data, let's run through it!

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_flat)
scaled_features = scaler.fit_transform(X_train_flat)

In [ ]:
X_train_new = pd.DataFrame(scaled_features,columns=X_train_flat.columns)
X_train_new.head()

### Create y_train_new dataset by encoding categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder = LabelEncoder()
y_train_new=labelencoder.fit_transform(y_train['surface'])
y_train_new=pd.Series(y_train_new)
print(labelencoder.classes_)

### Flattening X_test and applying Standard Scaling to our dataset

In [ ]:
X_test = pd.read_csv("../input/X_test.csv")

X_test_flat= flattenDataframe(X_test)
scaler.fit(X_test_flat)
scaled_features_train = scaler.fit_transform(X_test_flat)
X_test_new = pd.DataFrame(scaled_features_train,columns=X_test_flat.columns)
X_test_new.head()

### Tensorflow

In [ ]:
import tensorflow as tf

### Create a list of feature column objects using tf.feature.numeric_column() 

In [ ]:
feat_cols = []
for key in X_train_new.keys():
    feat_cols.append(tf.feature_column.numeric_column(key=key))
feat_cols

### Create an object called classifier which is a DNNClassifier from learn.
Set it to have 9 classes and a [20,40,20] hidden unit layer structure:

In [ ]:
steps=20000
n=9
Layers = [20,40,20]  
classifier = tf.estimator.DNNClassifier(
                                        hidden_units=Layers,
                                        n_classes=n,
                                        feature_columns=feat_cols)

### Now create a tf.estimator.pandas_input_fn that takes in your X_train, y_train, batch_size and set shuffle=True. 

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train_new,
                                                 y=y_train_new,
                                                batch_size=10,num_epochs=500,shuffle=True)

### Now train classifier to the input function 

In [ ]:
classifier.train(input_fn=input_func,steps=steps)

### Use the predict method from the classifier model to create predictions from X_test 

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(
      x=X_test_new,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [ ]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

In [ ]:
final_preds  = []
for pred in note_predictions:
    class_id=pred['class_ids'][0]
    final_preds.append(class_id)
    
set(final_preds)

### Create a dataframe from our predictions 

In [ ]:
df_pred= pd.DataFrame({'surface_code':final_preds})
df_pred.head()

### Decoding our surface_code and save the result to submission file

In [ ]:
df_pred['surface']=labelencoder.inverse_transform(df_pred['surface_code'])
sub['surface'] = df_pred['surface']
sub.to_csv("submission.csv",index=False)
sub.head()

In [ ]:
#train_eval_result = classifier.evaluate(input_fn=input_func)
#print("Training set accuracy: {accuracy}".format(**train_eval_result))

train_eval_result = classifier.evaluate(input_fn=input_func)
print("Training set accuracy: {accuracy}".format(**train_eval_result))
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-19T17:57:11Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-19-18:50:53
INFO:tensorflow:Saving dict for global step 20000: accuracy = 0.9007874, average_loss = 0.2592908, global_step = 20000, loss = 2.5929081
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 20000: 

**Training set accuracy: 0.9007874131202698**